In [13]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests


#### After import necessary package, using BeautifulSoup to obtain the table

In [14]:
wiki_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wiki_source = requests.get(wiki_url).text
soup = BeautifulSoup(wiki_source, 'lxml')
table = soup.tbody

#### Initialize the DataFrame

In [15]:
collist = ['PostalCode', 'Borough', 'Neighbourhood']
df_og = pd.DataFrame(columns = collist)

#### Some Helper Function

##### Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [16]:
def na_borough(row):
    return row[1] == 'Not assigned'

##### More than one neighborhood can exist in one postal code area. These two rows will be combined into one row with the neighborhoods separated with a comma

In [17]:
def combine_rows(row, df):
    if row[0] not in df['PostalCode'].values:
        return df.append(pd.Series(row, index = df.columns), ignore_index = True)
    target = df[df['PostalCode'] == row[0]].index[0]
    df.loc[target, 'Neighbourhood'] = (df[df['PostalCode'] == row[0]]['Neighbourhood'].values + ', ' + row[2])[0] 
    return df

##### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [18]:
def na_neighbourhood(row):
    if row[2] == 'Not assigned':
        row[2] = row[1]
    return row

#### Go through the Data and form the DataFrame

In [19]:
for line in table.find_all('tr'):
    tmp = []
    for item in line.find_all('td'):
        tmp.append(item.text.strip())
    if len(tmp) != 3:
        continue
    if na_borough(tmp):
        continue
    tmp = na_neighbourhood(tmp)
    df_og = combine_rows(tmp, df_og)

#### Read in geographical coordinates file

In [20]:
geo_url = 'http://cocl.us/Geospatial_data'
df_geo = pd.read_csv(geo_url)
df_geo.columns = ['PostalCode', 'Latitude', 'Longitude']
print (df_geo.head())

  PostalCode   Latitude  Longitude
0        M1B  43.806686 -79.194353
1        M1C  43.784535 -79.160497
2        M1E  43.763573 -79.188711
3        M1G  43.770992 -79.216917
4        M1H  43.773136 -79.239476


In [12]:
df_merged = df_og.join(df_geo.set_index('PostalCode'), on='PostalCode')
print (df_merged.head())

    PostalCode           Borough  \
98         M8X         Etobicoke   
99         M4Y  Downtown Toronto   
100        M7Y      East Toronto   
101        M8Y         Etobicoke   
102        M8Z         Etobicoke   

                                         Neighbourhood   Latitude  Longitude  
98       The Kingsway, Montgomery Road, Old Mill North  43.653654 -79.506944  
99                                Church and Wellesley  43.665860 -79.383160  
100  Business Reply Mail Processing Centre 969 Eastern  43.662744 -79.321558  
101  Humber Bay, King's Mill Park, Kingsway Park So...  43.636258 -79.498509  
102  Kingsway Park South West, Mimico NW, The Queen...  43.628841 -79.520999  


#### Merge two DataFrame